# 层次聚类

In [17]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
# mpl.rcParams['font.sans-serif'] = ['SimHei']
# mpl.rcParams['font.serif'] = ['SimHei']
from matplotlib.font_manager import FontProperties
font = FontProperties(fname=r"./Font/times.ttf", size=12)
import seaborn as sns

from scipy.cluster import hierarchy
from scipy.cluster import hierarchy
from scipy import cluster
import matplotlib.pyplot as plt
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签

plt.rc('font',family='Times New Roman')
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
plt.rcParams['savefig.dpi'] = 300 #图片像素
plt.rcParams['figure.dpi'] = 300 #分辨率
plt.rcParams['figure.figsize'] = (20.0, 20.0)
#plt.tick_params(labelsize=25)#改坐标轴刻度字体大小

# 需要设置的所有参数 #
read_root 和 save_root 表示数据读取和存储路径

默认层次聚类图存储路径为 read_root/cluster 下 all.png
默认单因素聚类图存储路径为 read_root/cluster 下 变量名.png
可通过savePicName变量自定义图片名

In [18]:
read_root = '.'               #数据存放路径
save_root = '.'               #图表存储路径

use_all = False               #是否用全部实验数据，如果True则use_d7变量不被考虑
if not use_all:
    use_d7 =False             #是否仅用第7天, 如果False则用第2天数据
use_wave = False               #True表示采用光谱，False表示采用理化值
if use_wave:
    use_wave_type = 'all'                # 聚类波长的挑选方式： 'all' 全波长; 'select' 特征波长； 'pca' PCA降维
    if use_wave_type == 'select':
        selected = [23,131,2,64,34,124]  # 如果用特征波长，则给定挑选波长的序号， eg. selected=[23,131,2,64,634,124]
    elif use_wave_type == 'pca':
        update_pca = False                # 是否更新pca的主成分数
        pca_n = 5                         # pca主成分数
else:
    phenotype = ['M156T536','M171T180','M188T542','M200T536','M232T33','M246T33','M254T59','M281T123','M289T53','M363T35','M405T389',
                 'M406T389','M506T56_2','M509T44','M542T535','M63T389','M65T389','M722T38'] #选择的表型参数，根据需要筛减

isScale=True                             # 是否归一化
measure='distance'                       #’distance‘ 表示采用MMD， ’mean‘表示采用平均值
            
savePicName_all = '4_24'                     # 设置存储的多因素聚类图名称
savePicName_one = ''                     # 设置存储的单因素聚类图名称
cmap_color = 'RdGy'

In [13]:
tmp = 'Accent, Accent_r, Blues, Blues_r, BrBG, BrBG_r, BuGn, BuGn_r, BuPu, BuPu_r, CMRmap, CMRmap_r, Dark2, Dark2_r, GnBu, GnBu_r, Greens, Greens_r, Greys, Greys_r, OrRd, OrRd_r, Oranges, Oranges_r, PRGn, PRGn_r, Paired, Paired_r, Pastel1, Pastel1_r, Pastel2, Pastel2_r, PiYG, PiYG_r, PuBu, PuBuGn, PuBuGn_r, PuBu_r, PuOr, PuOr_r, PuRd, PuRd_r, Purples, Purples_r, RdBu, RdBu_r, RdGy, RdGy_r, RdPu, RdPu_r, RdYlBu, RdYlBu_r, RdYlGn, RdYlGn_r, Reds, Reds_r, Set1, Set1_r, Set2, Set2_r, Set3, Set3_r, Spectral, Spectral_r, Wistia, Wistia_r, YlGn, YlGnBu, YlGnBu_r, YlGn_r, YlOrBr, YlOrBr_r, YlOrRd, YlOrRd_r, afmhot, afmhot_r, autumn, autumn_r, binary, binary_r, bone, bone_r, brg, brg_r, bwr, bwr_r, cividis, cividis_r, cool, cool_r, coolwarm, coolwarm_r, copper, copper_r, cubehelix, cubehelix_r, flag, flag_r, gist_earth, gist_earth_r, gist_gray, gist_gray_r, gist_heat, gist_heat_r, gist_ncar, gist_ncar_r, gist_rainbow, gist_rainbow_r, gist_stern, gist_stern_r, gist_yarg, gist_yarg_r, gnuplot, gnuplot2, gnuplot2_r, gnuplot_r, gray, gray_r, hot, hot_r, hsv, hsv_r, icefire, icefire_r, inferno, inferno_r, jet, jet_r, magma, magma_r, mako, mako_r, nipy_spectral, nipy_spectral_r, ocean, ocean_r, pink, pink_r, plasma, plasma_r, prism, prism_r, rainbow, rainbow_r, rocket, rocket_r, seismic, seismic_r, spring, spring_r, summer, summer_r, tab10, tab10_r, tab20, tab20_r, tab20b, tab20b_r, tab20c, tab20c_r, terrain, terrain_r, viridis, viridis_r, vlag, vlag_r, winter, winter_r'

In [14]:
# 在读取目录下要有这两个文件 biomass.xlsx和wavelength.npy
df = pd.read_excel(os.path.join(read_root, 'mmd-me.xlsx'))
wavename = np.load(os.path.join(read_root, 'wavelength.npy'))
d3 = df[df['time']==3]
d7 = df[df['time']==7]
if use_all:
    dfused = df
else:
    dfused = d7 if use_d7 else d3

In [15]:
categorylist = ['M156T536','M171T180','M188T542','M200T536','M232T33','M246T33','M254T59','M281T123','M289T53','M363T35',
                'M405T389','M406T389','M506T56_2','M509T44','M542T535','M63T389','M65T389','M722T38']

if use_wave:
    if use_wave_type == 'all':
        attributes = [i for i in range(432)]
    elif use_wave_type == 'select':
        attributes = selected
    else:
        attributes = ['p'+str(i) for i in range(5)]
else:
    attributes = phenotype

In [16]:
def Scale(q, p):
    len_qz = len(q)
    data = np.concatenate([q, p])
    for attr in range(data.shape[1]):
        Max = np.max(data[:, attr])
        Min = np.min(data[:, attr])
        data[:, attr] = (Max - data[:, attr]) / (Max - Min)
    return data[:len_qz], data[len_qz:]


def mmd_penalty_with_p(sample_qz, sample_pz, kernel='RBF', with_scale=True):
    '''
    p| * q_
    :param sample_qz:
    :param sample_pz:
    :param kernel:
    :return:
    '''
    if len(sample_qz.shape)==1:
        sample_qz = sample_qz[:, np.newaxis]
        sample_pz = sample_pz[:, np.newaxis]
        if with_scale:
            sample_qz, sample_pz = Scale(sample_qz, sample_pz)


    cross_pz = np.matmul(sample_pz, sample_pz.T)
    distances_pz = cross_pz.diagonal()[None, :] + cross_pz.diagonal()[:, None] - 2 * cross_pz

    cross_qz = np.matmul(sample_qz, sample_qz.T)
    distances_qz = cross_qz.diagonal()[None, :] + cross_qz.diagonal()[:, None] - 2 * cross_qz

    distances = cross_pz.diagonal()[:, None] + cross_qz.diagonal()[None, :] - 2 * np.matmul(sample_pz, sample_qz.T)

    # sample_pz = (sample_pz-Min)/(Max-Min)
    m = len(sample_pz)
    n = len(sample_qz)
    #   print('n', n)
    q_ = (np.ones(n)/n)[:, np.newaxis]
    p_ = (np.ones(m)/m)[:, np.newaxis]

    if kernel=='IMQ':
        # sigma2_p = 1
        # # print(q_.shape)
        # if config.mmd_pz == 'normal':
        #     Cbase = 2. * opts['zdim'] * sigma2_p
        # elif config.mmd_pz == 'sphere':
        #     Cbase = 2.
        # elif config.mmd_pz == 'uniform':
        #     Cbase = opts['zdim']
        # stat = 0.
        # for scale in [.1, .2, .5, 1., 2., 5., 10.]:
        #     C = Cbase * scale
        #     res1 = torch.sum( np.matmul(q_, q_.T) * C / (C + distances_qz))  # / (n**2 - n))
        #     res1 += torch.sum( np.matmul(p_, p_.T) * C / (C + distances_pz))  # / (m**2 - m))
        #     res2 = torch.sum( np.matmul(p_, q_.T) * C / (C + distances) * 2.)
        #     # res2 = torch.sum(res2) * 2. / (n * m)
        #     stat += res1 - res2
        pass
    elif kernel == 'RBF':
        sigma2_k = 8
        res1 = np.sum( np.exp(distances_qz / -2./sigma2_k) * np.matmul(q_, q_.T))
        res1 +=np.sum( np.exp(distances_pz / -2./sigma2_k) * np.matmul(p_, p_.T))
        res2 = np.sum( np.exp(distances / -2./ sigma2_k) * np.matmul(p_, q_.T)) * 2
        stat = res1 - res2
    return stat

def convert(df):
    attributes = ['M156T536','M171T180','M188T542','M200T536','M232T33','M246T33','M254T59','M281T123','M289T53','M363T35','M405T389','M406T389','M506T56_2','M509T44','M542T535','M63T389','M65T389','M722T38' 
                   ]
    df_ = pd.DataFrame()
    df_['category']=None
    df_['attr'] = None
    df_['value'] = None
    h = 0
    for i in range(len(df)):
        for a in attributes:
            df_.loc[str(h)] =[df.loc[i].category, a, df.loc[i][a]]
            h += 1
    return df_

def drawClusterMap(df, title, isSave=True):
    sns.set()
    if True:
        flights = df.set_index('category')
    else:
        flights_long = convert(df)
        flights = flights_long.pivot("category", "attr", "value")
    g = sns.clustermap(flights, fmt="d", cmap="RdBu")
    ax = g.ax_heatmap
    label_y = ax.get_yticklabels()
    plt.title(title,fontproperties=font)
    plt.setp(label_y, rotation=360, horizontalalignment='left',fontproperties=font)
    if isSave:
        if savePicName_all=='':
            plt.savefig(os.path.join(save_root, 'cluster/all.png'))
        else:
            plt.savefig(os.path.join(save_root, 'cluster/{}.png'.format(savePicName_all)))
    else:
        plt.show()

def drawg(df, attr):
    mpl.rcParams['font.sans-serif'] = ['SimHei']
    if isinstance(attr, int):
        attr = str(wavename[attr])
    df = df[['category',attr]]
    df = df.set_index('category')
    Z = hierarchy.linkage(df, method='ward', metric='euclidean')
    hierarchy.dendrogram(Z, labels=df.index,leaf_font_size=45,leaf_rotation=45,distance_sort='ascending')
    if savePicName_one=='':
        plt.savefig(os.path.join(save_root, 'cluster/{}.png'.format(attr)))
    else:
        plt.savefig(os.path.join(save_root, 'cluster/{}.png'.format(savePicName_one)))
def CalDiff(data, caltype='distance', with_scale=False,SaveDiff=False, SavePic=True):
    '''
    caltype: 'distance' or 'mean'
    '''
    result = {'category':[]}
    for attr in attributes:
        if use_wave and use_wave_type!='pca':
            result.update({str(wavename[attr]):[]})
        else:
            result.update({attr:[]})
    for cate in categorylist:
        data_cate = data[data['category']==cate]
        data_cate_ck = data_cate[data_cate['deal'].str.contains('ck')]
        data_cate_nl = data_cate[data_cate['deal'].str.contains('nacl')]
        result['category'].append(cate)

        for attr in attributes:
            if use_wave and use_wave_type!='pca':
                str_attr = str(wavename[attr])
                attr = 'v'+str(attr)
            else:
                str_attr = attr
                
#             if str_attr=='vis':
#                 attr = data.columns[19:19+432]
#             elif str_attr=='nir':
#                 attr = data.columns[451:451+200]
                
            data_cate_ck_attr = data_cate_ck[attr].values
            data_cate_nl_attr = data_cate_nl[attr].values
            data_cate_ck_attr = data_cate_ck_attr[~np.isnan(data_cate_ck_attr)]
            data_cate_nl_attr = data_cate_nl_attr[~np.isnan(data_cate_nl_attr)]
            if caltype=='distance':
                dist = mmd_penalty_with_p(data_cate_ck_attr, data_cate_nl_attr, with_scale=with_scale)
            else:
                dist = np.mean(data_cate_ck_attr)
            result[str_attr].append(dist)
    df = pd.DataFrame(result)
    if SaveDiff:
        df.to_excel(os.path.join(save_root, 'dist.xlsx'))
    drawClusterMap(df, title=caltype, isSave=SavePic)
    return df



In [7]:
result = CalDiff(dfused, caltype=measure, with_scale=isScale,SaveDiff=True, SavePic=True)

ValueError: zero-size array to reduction operation maximum which has no identity

# 单因素层次聚类 #
选择attributes中的某个变量名进行聚类，需要先运行上一句得到df_

In [ ]:
attributes

In [ ]:
#drawg的第二个变量为要聚类的变量名，必须在attributes中已有;若是光谱则输入序号
drawg(result, 2)

# 更新pca列，大概不需要 #

In [ ]:
if update_pca:
    from sklearn.decomposition import PCA  
    assert(pca_n>0, '主成分数需要大于0')
    pca = PCA(n_components=pca_n)
    reduced_N=pca.fit_transform(N)
    for i in range(5):
        df['p'+str(i)] = reduced_N[:,i]
    df.to_excel(os.path.join(read_root, '/biomass.xlsx'))

In [ ]:
import sys
sys.executable